Step 1: Find the active buoys closest to Happisburgh, Norfolk - a place that struggles with coastal erosion.

https://www.ndbc.noaa.gov/
buoy nr: 62144 - Clipper AWS

Step 2: Fetch data from the closest buoy 

In [3]:
import pandas as pd

# Fetch real-time data for the closest buoy
buoy_id = 62144
data_url = f"https://www.ndbc.noaa.gov/data/realtime2/{buoy_id}.txt"

# Load buoy data
try:
    data = pd.read_csv(data_url, delim_whitespace=True, skiprows=[1], na_values=["MM"])
    print(data.head())
except Exception as e:
    print(f"Error fetching data for buoy {buoy_id}: {e}")

C:\Users\sonia\AppData\Local\Temp\ipykernel_17604\896513729.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(data_url, delim_whitespace=True, skiprows=[1], na_values=["MM"])


    #YY  MM  DD  hh  mm  WDIR  WSPD  GST  WVHT  DPD  APD  MWD    PRES  ATMP  \
0  2025   1  16  16   0   200   6.2  NaN   0.6  NaN  NaN  NaN  1033.4   5.4   
1  2025   1  16  15   0   200   5.1  NaN   0.6  NaN  NaN  NaN  1033.0   5.2   
2  2025   1  16  14   0   220   4.1  NaN   0.6  NaN  NaN  NaN  1033.2   5.1   
3  2025   1  16  13   0   220   5.1  NaN   0.6  NaN  NaN  NaN  1033.6   5.1   
4  2025   1  16  12   0   200   5.1  NaN   0.5  NaN  NaN  NaN  1034.1   4.6   

   WTMP  DEWP  VIS  PTDY  TIDE  
0   NaN   5.4  1.0  -0.2   NaN  
1   NaN   5.2  1.0  -1.1   NaN  
2   NaN   5.1  1.0  -1.0   NaN  
3   NaN   5.1  1.0  -0.4   NaN  
4   NaN   4.6  2.0  -0.1   NaN  


Step 3: Extract data to guide initial params

In [4]:
# Drop rows with missing wave height
data = data.dropna(subset=["WVHT"])

wave_height = data["WVHT"].mean()  # Significant wave height
wave_speed = data["WSPD"].mean()  

# Output parameters
print(f"WAVE_HEIGHT: {wave_height}") # m
print(f"WAVE_SPEED: {wave_speed}") # m/s

WAVE_HEIGHT: 0.58
WAVE_SPEED: 5.363999999999999


Based on ,.. wave period near Norfolk was about 9s for 2012-2013

In [5]:
import numpy as np

# Constants
g = 9.81  # gravity (m/s^2)
rho = 1025  # water density (kg/m^3)
T = 4.81  # representative wave period (s)

# 1. Wave Frequency
wavelength = (g * T**2) / (2 * np.pi)
wave_freq = wave_speed / wavelength

# 2. Wave Decay (empirical coefficient)
wave_decay = 0.6  # placeholder, adjust based on literature

# 3. Wave Cutoff
beta = 0.1  # empirical constant
wave_cutoff = beta * wave_height

# 4. Wave Retreat Coefficient
wave_retreat_coeff = rho * g * wave_height**2 * wave_speed

# 5. Sediment Transport Coefficients
shear_velocity = wave_speed * 0.05  # estimate, 5% of wave speed
shear_stress = rho * shear_velocity**2
sand_pull = shear_stress**1.5
ground_pull = sand_pull * 0.5  # assume ground pull is half of sand pull

# Output results
print(f"Wave Frequency: {wave_freq:.3f} Hz")
print(f"Wave Decay: {wave_decay}")
print(f"Wave Cutoff: {wave_cutoff:.3f} m")
print(f"Wave Retreat Coefficient: {wave_retreat_coeff:.3f}")
print(f"Sand Pull: {sand_pull:.3f}")
print(f"Ground Pull: {ground_pull:.3f}")

Wave Frequency: 0.148 Hz
Wave Decay: 0.6
Wave Cutoff: 0.058 m
Wave Retreat Coefficient: 18144.192
Sand Pull: 633.085
Ground Pull: 316.543


: 